In [ ]:
#| default_exp utilities.sparse

# utilities.sparse

> Extra functionality for [sparse](https://sparse.pydata.org) and [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html)

In [ ]:
#| hide
from fastcore.test import test_eq
from pylgs.utilities.testing import test_array

In [ ]:
#| export
import itertools as it

from fastcore.meta import delegates

import numpy as np
from numpy import array, ndarray
import scipy.sparse as sps
from scipy.sparse import sparray
import scipy.linalg as spl
import scipy.sparse.linalg as spa
from sparse import COO, GCXS, SparseArray
import sparse as ss

### sparse -

In [ ]:
#|export
def sparse2d(a):
    """Create a 2D sparse array in specified format."""
    if isinstance(a, SparseArray): a = a.to_scipy_sparse()
    return sps.csr_array(a)

In [ ]:
arrays = [np.identity(2), sps.eye(2), ss.eye(2), ss.eye(2, format='gcxs')]
[sparse2d(args) for args in arrays]

[<2x2 sparse array of type '<class 'numpy.float64'>'
 	with 2 stored elements in Compressed Sparse Row format>,
 <2x2 sparse array of type '<class 'numpy.float64'>'
 	with 2 stored elements in Compressed Sparse Row format>,
 <2x2 sparse array of type '<class 'numpy.float64'>'
 	with 2 stored elements in Compressed Sparse Row format>,
 <2x2 sparse array of type '<class 'numpy.float64'>'
 	with 2 stored elements in Compressed Sparse Row format>]

In [ ]:
#|hide
test_eq([a.__class__.__name__ for a in _], ['csr_array', 'csr_array', 'csr_array', 'csr_array'])

In [ ]:
#|export
@delegates(COO, but='data')
def sparse(a, format='coo', **kwargs)->SparseArray:
    """Create an N-D sparse array in specified format."""
    if format == 'gcxs': cls = GCXS
    elif format == 'coo': cls = COO
    if isinstance(a, ndarray): return cls.from_numpy(a)
    if sps.issparse(a): return cls.from_scipy_sparse(a)
    if isinstance(a, SparseArray): return cls(a)
    return cls(COO(*a, **kwargs)) # Assume a is coords and data in COO format

In [ ]:
arrays = [np.identity(2), sps.eye(2), ss.eye(2), [[[0, 1], [0, 1]], [1., 1.]]]
[sparse(*args) for args in it.product(arrays, ['coo', 'gcxs'])]

[<COO: shape=(2, 2), dtype=float64, nnz=2, fill_value=0.0>,
 <GCXS: shape=(2, 2), dtype=float64, nnz=2, fill_value=0.0, compressed_axes=(0,)>,
 <COO: shape=(2, 2), dtype=float64, nnz=2, fill_value=0.0>,
 <GCXS: shape=(2, 2), dtype=float64, nnz=2, fill_value=0, compressed_axes=(0,)>,
 <COO: shape=(2, 2), dtype=float64, nnz=2, fill_value=0.0>,
 <GCXS: shape=(2, 2), dtype=float64, nnz=2, fill_value=0.0, compressed_axes=(0,)>,
 <COO: shape=(2, 2), dtype=float64, nnz=2, fill_value=0.0>,
 <GCXS: shape=(2, 2), dtype=float64, nnz=2, fill_value=0.0, compressed_axes=(0,)>]

In [ ]:
#|hide
test_eq([a.__class__.__name__ for a in _], ['COO', 'GCXS', 'COO', 'GCXS', 'COO', 'GCXS', 'COO', 'GCXS'])

### sparse_identity -

In [ ]:
#|export
def sparse2d_identity(n):
    return sps.eye_array(n)

In [ ]:
sparse2d_identity(2)

<2x2 sparse array of type '<class 'numpy.float64'>'
	with 2 stored elements (1 diagonals) in DIAgonal format>

In [ ]:
#|export
def sparse_identity(n, format='coo')->SparseArray:
    return ss.eye(n, format=format)

In [ ]:
[sparse_identity(2, format=fmt) for fmt in ['coo', 'gcxs']]

[<COO: shape=(2, 2), dtype=float64, nnz=2, fill_value=0.0>,
 <GCXS: shape=(2, 2), dtype=float64, nnz=2, fill_value=0.0, compressed_axes=(0,)>]

### sparse_diag -

In [ ]:
#|export
def sparse2d_diag(diags, offsets=0):
    return sps.diags_array(diags, offsets=offsets)

In [ ]:
sparse2d_diag([1, 1])

<2x2 sparse array of type '<class 'numpy.float64'>'
	with 2 stored elements (1 diagonals) in DIAgonal format>

In [ ]:
#| export
def sparse_diag(diag, format='coo')->SparseArray:
    return sparse(sparse2d_diag(diag), format=format)

In [ ]:
sparse_diag([1, 1])

Format,coo
Data Type,float64
Shape,"(2, 2)"
nnz,2
Density,0.5
Read-only,True
Size,32
Storage ratio,1.00


### sparse_toeplitz -

In [ ]:
#| export
def dense_toeplitz(
    diags:ndarray, # Value on each diagonal, starting at lower left.
):
    """Return a toeplitz array given the value on each diagonal (starting at lower left)."""
    n = (len(diags) - 1) // 2
    return spl.toeplitz(diags[:n+1][::-1], diags[n:])

In [ ]:
dense_toeplitz([1, 2, 3])

array([[2, 3],
       [1, 2]])

In [ ]:
#| export
def sparse_toeplitz(
    diags:ndarray, # Value on each diagonal, starting at lower left.
    format='coo'
)->SparseArray:
    """Return a toeplitz sparse array in specified format given the value on each diagonal (starting at lower left)."""
    return sparse(dense_toeplitz(diags), format=format)

In [ ]:
sparse_toeplitz([1, 2, 3, 4, 5])

Format,coo
Data Type,int64
Shape,"(3, 3)"
nnz,9
Density,1.0
Read-only,True
Size,216
Storage ratio,3.00


In [ ]:
_.todense()

array([[3, 4, 5],
       [2, 3, 4],
       [1, 2, 3]])

In [ ]:
#| hide
test_array('utilities', 'sparse_toeplitz', _)

In [ ]:
#| export
def sparse2d_kronecker_matrix(n, k=0):
    """The nxn sparse kronecker matrix delta_(i,j-k) in specified format."""
    return sparse2d_diag(np.ones(n - abs(k)), offsets=k)

In [ ]:
sparse2d_kronecker_matrix(3, 2)

<3x3 sparse array of type '<class 'numpy.float64'>'
	with 1 stored elements (1 diagonals) in DIAgonal format>

In [ ]:
_.todense()

array([[0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
#| export
def sparse_kronecker_matrix(n, k=0, format='coo')->SparseArray:
    """The nxn sparse kronecker matrix delta_(i,j-k) in specified format."""
    return sparse(sparse2d_kronecker_matrix(n, k), format=format)

In [ ]:
sparse_kronecker_matrix(3, 2)

Format,coo
Data Type,float64
Shape,"(3, 3)"
nnz,1
Density,0.1111111111111111
Read-only,True
Size,16
Storage ratio,0.22


In [ ]:
#| hide
test_array('utilities', 'sparse_kron', _)

In [ ]:
#| export 
def kron(a, b, *rest):
    if not rest: 
        if sps.issparse(a) and sps.issparse(b): return sps.kron(a, b, format='csr')
        return np.kron(a, b)
    return kron(kron(a, b), *rest)

In [ ]:
kron(np.eye(2), np.eye(2), np.eye(2))

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [ ]:
#| hide
test_array('utilities', 'kron_1', _)

In [ ]:
kron(*[sparse2d_identity(2)] * 3)

<8x8 sparse array of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [ ]:
#| hide
test_array('utilities', 'kron_1', _.todense())

### sparse2d_rand -

In [ ]:
#|export
sparse2d_rand = sps.random_array

In [ ]:
sparse2d_rand([2, 3])

<2x3 sparse array of type '<class 'numpy.float64'>'
	with 0 stored elements in COOrdinate format>

### spilu -

In [ ]:
#|export
spilu = spa.spilu

### restrict_bandwidth -

In [ ]:
#| export
def restrict_bandwidth(a_sparray:sparray, width):
    """Make a scipy sparse array banded by setting all elements outside the bandwidth to zero."""
    result = a_sparray.copy()
    i, j = result.nonzero()
    result.data[np.abs(i - j) > width] = 0.
    result.eliminate_zeros()
    return result

In [ ]:
restrict_bandwidth(sps.csr_array(np.ones((5, 5))), 1)

<5x5 sparse array of type '<class 'numpy.float64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [ ]:
_.toarray()

array([[1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0.],
       [0., 1., 1., 1., 0.],
       [0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.]])

In [ ]:
#| hide
test_array('utilities', 'restrict_bandwidth', _)

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()